<a href="https://colab.research.google.com/github/bessabra/PRO2PRO/blob/main/cacopospro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [257]:
import pandas as pd
from datetime import datetime, timedelta


base  = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Progressão Professores v20.12/Versões v20.12/CACOPOSPRO V13.12.2.csv')
base2 = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Progressão Professores v20.12/Versões v20.12/Resultados/baseAnalisev20.12.4.csv')


Retirada de colunas desnecessáras.

In [258]:
base2.drop('Unnamed: 0.1',  axis = 1, inplace=True)
base2.drop('Unnamed: 0',    axis = 1, inplace=True)
base2.drop('_merge',        axis = 1, inplace=True)
base2.drop('DRstatus',      axis = 1, inplace=True)
base2.drop('DR5status',     axis = 1, inplace=True)
base2.drop('REQUERIMENTO',  axis = 1, inplace=True)

base.drop('Órgão',    axis = 1, inplace=True)
base.drop('SIAPECAD', axis = 1, inplace=True)

Redefinição de tipos das variáveis.

In [259]:
base['CARGO INICIAL']     = pd.to_datetime(base['CARGO INICIAL'], errors="coerce")
base['CARGO FINAL']       = pd.to_datetime(base['CARGO FINAL'], errors="coerce")
base['VIGENCIA INICIAL']  = pd.to_datetime(base['VIGENCIA INICIAL'], errors="coerce")
base['VIGENCIA FINAL']    = pd.to_datetime(base['VIGENCIA FINAL'], errors="coerce")

base2['OBITO']                  = pd.to_datetime(base2['OBITO'], errors="coerce")
base2['INATIVIDADE']            = pd.to_datetime(base2['INATIVIDADE'], errors="coerce")
base2['DATA DO REQUERIMENTO']   = pd.to_datetime(base2['DATA DO REQUERIMENTO'], errors="coerce")
base2['DR5']                    = pd.to_datetime(base2['DR5'], errors="coerce")

Filtrando CPF, SIAPE e NOME presentes em CACOPOSPRO V13.12.2 e baseAnalisev19.12.

In [260]:
servEncontradosCaco = base.merge(
    base2, on=['CPF', 'SIAPE', 'NOME'], 
    # how='outer',
    how='inner',
    suffixes=['', '_'], 
    indicator=True
    )

servEncontradosCaco['DiasDiffDR'] = (servEncontradosCaco['VIGENCIA INICIAL'] - servEncontradosCaco['DATA DO REQUERIMENTO']).dt.days.abs()

servEncontradosCacoDR = servEncontradosCaco.sort_values(by=['NOME', 'DiasDiffDR'])
servEncontradosCacoDR = servEncontradosCacoDR.groupby('NOME').first().reset_index()

servEncontradosCacoDR = servEncontradosCacoDR.rename(
    columns = {'NIVEL POR DATA':'NivelEmDR'}
    )
servEncontradosCaco = servEncontradosCaco.rename(
    columns = {'_merge':'MergeGeral'}
    )
servEncontradosCacoDR = servEncontradosCacoDR.rename(
    columns = {'_merge':'MergeDR'}
    )

servEncontradosCaco = servEncontradosCaco.merge(
    servEncontradosCacoDR, on=['CPF', 'SIAPE', 'NOME'], 
    # how='outer',
    how='inner',
    suffixes=['', '_'], 
    indicator=True
    )

In [261]:
servEncontradosCaco.drop('CARGO INICIAL', axis = 1, inplace=True)
servEncontradosCaco.drop('CARGO FINAL',    axis = 1, inplace=True)
servEncontradosCaco.drop('CARGO_',    axis = 1, inplace=True)
servEncontradosCaco.drop('CLASSE',    axis = 1, inplace=True)

servEncontradosCaco.drop('NIVEL', axis = 1, inplace=True)
servEncontradosCaco.drop('CONDIÇÃO',    axis = 1, inplace=True)
servEncontradosCaco.drop('MergeGeral',    axis = 1, inplace=True)
# servEncontradosCaco.drop('VIGENCIA INICIAL_',    axis = 1, inplace=True)
# servEncontradosCaco.drop('VIGENCIA FINAL_', axis = 1, inplace=True)
servEncontradosCaco.drop('SITUAÇÃO_',    axis = 1, inplace=True)
servEncontradosCaco.drop('OBITO_',    axis = 1, inplace=True)
servEncontradosCaco.drop('INATIVIDADE_',    axis = 1, inplace=True)

servEncontradosCaco.drop('PROCESSO SEI_', axis = 1, inplace=True)
servEncontradosCaco.drop('ESTADO_',    axis = 1, inplace=True)
# servEncontradosCaco.drop('MergeGeral',    axis = 1, inplace=True)
servEncontradosCaco.drop('DATA DO REQUERIMENTO_',    axis = 1, inplace=True)
servEncontradosCaco.drop('DESCRIÇÃO_', axis = 1, inplace=True)
servEncontradosCaco.drop('CARGO__',    axis = 1, inplace=True)
servEncontradosCaco.drop('CLASSE_',    axis = 1, inplace=True)
servEncontradosCaco.drop('NIVEL_',    axis = 1, inplace=True)

servEncontradosCaco.drop('CONDIÇÃO_', axis = 1, inplace=True)
servEncontradosCaco.drop('DR5_',    axis = 1, inplace=True)
servEncontradosCaco.drop('MergeDR',    axis = 1, inplace=True)
servEncontradosCaco.drop('DiasDiffDR_',    axis = 1, inplace=True)
servEncontradosCaco.drop('_merge', axis = 1, inplace=True)

servEncontradosCaco.drop('CARGO INICIAL_', axis = 1, inplace=True)
servEncontradosCaco.drop('CARGO FINAL_',    axis = 1, inplace=True)

In [262]:
# servEncontradosCaco.info()
# servEncontradosCaco.to_excel('servEncontradosCacoDRv21.12.3.xlsx')

In [263]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-101')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-101')), 'COD_NIVELDR'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-102')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-102')), 'COD_NIVELDR'] = 2
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-201')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-201')), 'COD_NIVELDR'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-202')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-202')), 'COD_NIVELDR'] = 4
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-301')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-301')), 'COD_NIVELDR'] = 5
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-302')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-302')), 'COD_NIVELDR'] = 6
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-303')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-303')), 'COD_NIVELDR'] = 7
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-304')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-304')), 'COD_NIVELDR'] = 8
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-401')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-401')), 'COD_NIVELDR'] = 9
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-402')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-402')), 'COD_NIVELDR'] = 10
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-403')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-403')), 'COD_NIVELDR'] = 11
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '709-NS-D-404')|(servEncontradosCaco['NivelEmDR'] == '707-NS-D-404')), 'COD_NIVELDR'] = 12

#--------------------------------------------------------------------------------------------------------------------------------------------

servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-101')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-101')), 'COD_NIVELDR'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-102')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-102')), 'COD_NIVELDR'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-103')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-103')), 'COD_NIVELDR'] = 2
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-104')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-104')), 'COD_NIVELDR'] = 2


servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-201')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-201')), 'COD_NIVELDR'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-202')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-202')), 'COD_NIVELDR'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-203')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-203')), 'COD_NIVELDR'] = 4
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-204')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-204')), 'COD_NIVELDR'] = 4
    
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-301')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-301')), 'COD_NIVELDR'] = 5
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-302')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-302')), 'COD_NIVELDR'] = 6
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-303')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-303')), 'COD_NIVELDR'] = 7
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-304')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-304')), 'COD_NIVELDR'] = 8
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-401')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-401')), 'COD_NIVELDR'] = 9
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-402')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-402')), 'COD_NIVELDR'] = 10
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-403')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-403')), 'COD_NIVELDR'] = 11
servEncontradosCaco.loc[
    ((servEncontradosCaco['NivelEmDR'] == '704-NS-D-404')|(servEncontradosCaco['NivelEmDR'] == '702-NA-D-404')), 'COD_NIVELDR'] = 12

In [264]:
servEncontradosCaco.loc[
    (servEncontradosCaco['VIGENCIA INICIAL_'] > servEncontradosCaco['DATA DO REQUERIMENTO']), 'COD_NIVELDR'] = servEncontradosCaco['COD_NIVELDR'] - 1

In [265]:
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 1), 'NIVEL'] = 101
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 2), 'NIVEL'] = 102
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 3), 'NIVEL'] = 201
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 4), 'NIVEL'] = 202
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 5), 'NIVEL'] = 301
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 6), 'NIVEL'] = 302
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 7), 'NIVEL'] = 303
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 8), 'NIVEL'] = 304
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 9), 'NIVEL'] = 401
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 10), 'NIVEL'] = 402
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] == 11), 'NIVEL'] = 403
servEncontradosCaco.loc[
    (servEncontradosCaco['COD_NIVELDR'] >= 12), 'NIVEL'] = 404

# servEncontradosCaco.to_excel('CADE.xlsx')

Calculando a diferença entre DR5 e as datas de VIGENCIA INICIAL de cada Código de Carreira e Classe, por servidor. 

Em seguida, uma nova base é criada filtrada com a data e o Código de Carreira e Classe de cada servidor.

In [266]:
servEncontradosCaco['DiasDiff'] = (servEncontradosCaco['VIGENCIA INICIAL'] - servEncontradosCaco['DR5']).dt.days.abs()

servEncontradosCaco = servEncontradosCaco.sort_values(by=['NOME', 'DiasDiff'])
servEncontradosCaco = servEncontradosCaco.groupby('NOME').first().reset_index()

Calculando (DTPi - DR5), para identificar o valor da diferença e filtrar a decisão.

In [267]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
    & ((servEncontradosCaco['CARGO'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')|(servEncontradosCaco['CARGO'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF <= 1'

servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
    & ((servEncontradosCaco['CARGO'] == '709001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO')|(servEncontradosCaco['CARGO'] == '704001 PROFESSOR DO ENSINO BASICO EX-TERRITORIO'))), 'REGRA'] = 'EBF > 1' 

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year <= 1) 
     & ((servEncontradosCaco['CARGO'] == '707001 PROFESSOR ENS BASICO TECN TECNOL')|(servEncontradosCaco['CARGO'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT <= 1'

servEncontradosCaco.loc[
     ((servEncontradosCaco['VIGENCIA INICIAL'].dt.year - servEncontradosCaco['DR5'].dt.year > 1) 
     & ((servEncontradosCaco['CARGO'] == '707001 PROFESSOR ENS BASICO TECN TECNOL') |(servEncontradosCaco['CARGO'] == '702001 PROF DO ENSINO BASICO TEC,TECNOLOGICO'))), 'REGRA'] = 'EBTT > 1'

Tabelas de conversão de Código de Carreira e Classe (object) para um código de valor entre 1 - 12 (inteiro), isso facilitará o incremento.

In [268]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-101')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-101')), 'COD_NIVEL'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-102')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-102')), 'COD_NIVEL'] = 2
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-201')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-201')), 'COD_NIVEL'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-202')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-202')), 'COD_NIVEL'] = 4
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-301')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-301')), 'COD_NIVEL'] = 5
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-302')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-302')), 'COD_NIVEL'] = 6
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-303')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-303')), 'COD_NIVEL'] = 7
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-304')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-304')), 'COD_NIVEL'] = 8
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-401')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-401')), 'COD_NIVEL'] = 9
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-402')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-402')), 'COD_NIVEL'] = 10
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-403')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-403')), 'COD_NIVEL'] = 11
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '709-NS-D-404')|(servEncontradosCaco['NIVEL POR DATA'] == '707-NS-D-404')), 'COD_NIVEL'] = 12

#--------------------------------------------------------------------------------------------------------------------------------------------

servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-101')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-101')), 'COD_NIVEL'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-102')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-102')), 'COD_NIVEL'] = 1
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-103')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-103')), 'COD_NIVEL'] = 2
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-104')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-104')), 'COD_NIVEL'] = 2


servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-201')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-201')), 'COD_NIVEL'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-202')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-202')), 'COD_NIVEL'] = 3
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-203')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-203')), 'COD_NIVEL'] = 4
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-204')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-204')), 'COD_NIVEL'] = 4
    
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-301')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-301')), 'COD_NIVEL'] = 5
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-302')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-302')), 'COD_NIVEL'] = 6
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-303')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-303')), 'COD_NIVEL'] = 7
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-304')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-304')), 'COD_NIVEL'] = 8
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-401')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-401')), 'COD_NIVEL'] = 9
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-402')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-402')), 'COD_NIVEL'] = 10
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-403')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-403')), 'COD_NIVEL'] = 11
servEncontradosCaco.loc[
    ((servEncontradosCaco['NIVEL POR DATA'] == '704-NS-D-404')|(servEncontradosCaco['NIVEL POR DATA'] == '702-NA-D-404')), 'COD_NIVEL'] = 12

In [269]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['VIGENCIA INICIAL'] > servEncontradosCaco['DR5'])&(servEncontradosCaco['COD_NIVEL'] > 1)), 'COD_NIVEL'] = servEncontradosCaco['COD_NIVEL'] - 1


Realização do incremento de acordo com o filtro da diferença entre DTPi - DR5.


In [270]:
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 3
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBF > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 4


servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT <= 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 2
servEncontradosCaco.loc[
    (servEncontradosCaco['REGRA'] == 'EBTT > 1'), 'INCREMENTO_CNR'] = servEncontradosCaco['COD_NIVEL'] + 3

Conversão do código de valores incrementados de acordo com a regra de diferença entre DTPi - DR5.

In [271]:
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 1), 'CNR'] = 101
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 2), 'CNR'] = 102
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 3), 'CNR'] = 201
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 4), 'CNR'] = 202
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 5), 'CNR'] = 301
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 6), 'CNR'] = 302
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 7), 'CNR'] = 303
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 8), 'CNR'] = 304
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 9), 'CNR'] = 401
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 10), 'CNR'] = 402
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] == 11), 'CNR'] = 403
servEncontradosCaco.loc[
    (servEncontradosCaco['INCREMENTO_CNR'] >= 12), 'CNR'] = 404

Descrição do resultado da análise.

In [272]:
servEncontradosCaco.loc[
    ((servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL']) 
    |(servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL'])), 'RESULTADO'] = 'A DECIDIR'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL']), 'RESULTADO'] = 'INDEFERIDO'

servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] < servEncontradosCaco['NIVEL']), 'CNRstatus'] = 'O CNR calculado é MENOR que o encontrado na Data do Requerimento'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] > servEncontradosCaco['NIVEL']), 'CNRstatus'] = 'O CNR calculado é MAIOR que o encontrado na Data do Requerimento'
servEncontradosCaco.loc[
    (servEncontradosCaco['CNR'] == servEncontradosCaco['NIVEL']), 'CNRstatus'] = 'O CNR calculado é IGUAL ao encontrado na Data do Requerimento'

Geração do xlsx com os resultados.

In [273]:
servEncontradosCaco.to_excel('ResultadoComplexo21.12.15.xlsx')